## 🎨 The Complete Picture

Here's what we've built across all notebooks:

```
┌─────────────────────────────────────────────────────────────┐
│                    RAG PIPELINE                             │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   📄 PDFs → ✂️ Chunks → 🔢 Embeddings → 🗄️ ChromaDB        │
│   (Nb 1)    (Nb 2)      (Nb 3)          (Nb 4)             │
│                                                             │
│   ┌─────────────────────────────────────────────────────┐  │
│   │              THIS NOTEBOOK (Nb 5)                   │  │
│   │                                                     │  │
│   │   Question → Search → Context → AI → Answer        │  │
│   └─────────────────────────────────────────────────────┘  │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

## Step 1: Set Up Everything

In [ ]:
# Complete setup - all previous steps in one cell
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import os
import shutil

# Set up paths
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_folder = os.path.join(project_root, 'data')
chroma_path = os.path.join(os.getcwd(), 'chroma_db')

print("🚀 Setting up RAG Pipeline...\n")

# Step 1: Load PDFs
print("📄 Step 1: Loading PDFs...")
all_pages = []
for pdf_name in os.listdir(data_folder):
    if pdf_name.endswith('.pdf'):
        loader = PyPDFLoader(os.path.join(data_folder, pdf_name))
        all_pages.extend(loader.load())
print(f"   ✅ Loaded {len(all_pages)} pages")

# Step 2: Create chunks
print("✂️ Step 2: Creating chunks...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(all_pages)
print(f"   ✅ Created {len(chunks)} chunks")

# Step 3: Load embeddings
print("🔢 Step 3: Loading embeddings model...")
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)
print("   ✅ Embeddings ready")

# Step 4: Create vector store
print("🗄️ Step 4: Creating ChromaDB...")
if os.path.exists(chroma_path):
    shutil.rmtree(chroma_path)

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=chroma_path
)
print(f"   ✅ Vector store ready ({len(chunks)} chunks)")

print("\n" + "="*60)
print("🎉 Pipeline ready! Now let's connect the AI...")

---

## Step 2: Connect to Groq API (AI Model)

In [ ]:
# Connect to Groq - a fast, free AI API
from langchain_groq import ChatGroq
from dotenv import load_dotenv

# Load API key from .env file
load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

if not api_key or api_key == "your_groq_api_key_here":
    print("⚠️ Please set up your Groq API key!")
    print("\n📝 Steps:")
    print("   1. Go to: https://console.groq.com/keys")
    print("   2. Create a free account")
    print("   3. Generate an API key")
    print("   4. Create a .env file in the project root with:")
    print("      GROQ_API_KEY=your_actual_key_here")
    print("   5. Run this cell again")
else:
    print("🤖 Connecting to Groq API...")
    
    llm = ChatGroq(
        model="llama-3.1-8b-instant",
        temperature=0.3,
        api_key=api_key
    )
    
    print("✅ Connected!")
    print("   Model: llama-3.1-8b-instant")
    print("   Speed: ~2-5 seconds per response! 🚀")
    
    # Quick test
    print("\n⏳ Testing connection...")
    response = llm.invoke("Say 'RAG is ready!' in 4 words.")
    print(f"✅ AI says: {response.content}")

### 💡 About Groq

**Groq** provides extremely fast AI inference:

| Provider | Speed | Cost |
|----------|-------|------|
| Local Ollama | 60-120 sec | Free |
| OpenAI | 3-8 sec | ~$0.002/query |
| **Groq** | **2-5 sec** | **Free tier!** |

We use `llama-3.1-8b-instant` - a fast, capable model.

---

## Step 3: Create the RAG Function

In [ ]:
def ask_my_documents(question, num_chunks=3, show_sources=True):
    """
    Ask a question about your PDF documents using RAG.
    
    Args:
        question: Your question as a string
        num_chunks: How many document chunks to retrieve (default: 3)
        show_sources: Whether to print source information (default: True)
    
    Returns:
        Dictionary with 'answer' and 'sources'
    """
    
    # ========================================
    # STEP A: RETRIEVAL - Find relevant chunks
    # ========================================
    docs = vectorstore.similarity_search(question, k=num_chunks)
    
    # ========================================
    # STEP B: Build context from chunks
    # ========================================
    context = ""
    sources = []
    
    for doc in docs:
        context += doc.page_content + "\n\n"
        source_info = {
            'file': doc.metadata.get('source', 'Unknown').split('/')[-1],
            'page': doc.metadata.get('page', '?')
        }
        sources.append(source_info)
    
    # ========================================
    # STEP C: Create prompt for AI
    # ========================================
    prompt = f"""You are a helpful research assistant. Answer the question based ONLY on the provided context.
If the answer is not in the context, say "I don't have enough information to answer that based on the provided documents."

Context from documents:
---
{context}
---

Question: {question}

Answer:"""
    
    # ========================================
    # STEP D: GENERATION - Get AI response
    # ========================================
    response = llm.invoke(prompt)
    answer = response.content if hasattr(response, 'content') else str(response)
    
    # ========================================
    # STEP E: Return results
    # ========================================
    return {
        'answer': answer.strip(),
        'sources': sources,
        'context': context  # Included for transparency
    }

print("✅ RAG function created!")
print("\n📖 Usage:")
print('   result = ask_my_documents("Your question here")')
print('   print(result["answer"])')

### 💡 Understanding the RAG Function

```
Question: "What is qualitative coding?"
              │
              ▼
┌─────────────────────────────────────┐
│ STEP A: RETRIEVAL                   │
│ Search ChromaDB for similar chunks  │
└─────────────────────────────────────┘
              │
              ▼
┌─────────────────────────────────────┐
│ STEP B: BUILD CONTEXT               │
│ Combine relevant chunks into text   │
└─────────────────────────────────────┘
              │
              ▼
┌─────────────────────────────────────┐
│ STEP C: CREATE PROMPT               │
│ "Based on this context, answer..."  │
└─────────────────────────────────────┘
              │
              ▼
┌─────────────────────────────────────┐
│ STEP D: GENERATION                  │
│ Send to AI (Groq) → Get answer      │
└─────────────────────────────────────┘
              │
              ▼
        Answer + Sources
```

---

## Step 4: Ask Your First Question! 🎉

In [ ]:
import time

# Your question
my_question = "What is qualitative data coding and why is it important?"

print("="*60)
print(f"❓ Question: {my_question}")
print("="*60)
print("\n⏳ Processing...\n")

# Time the response
start = time.time()
result = ask_my_documents(my_question)
duration = time.time() - start

# Display the answer
print("💬 ANSWER:")
print("-"*60)
print(result['answer'])
print("-"*60)

# Show sources
print("\n📚 Sources used:")
for i, src in enumerate(result['sources'], 1):
    print(f"   {i}. {src['file']}, Page {src['page']}")

print(f"\n⏱️ Response time: {duration:.1f} seconds")

---

## Step 5: Try More Questions!

In [ ]:
# Helper function for clean output
def ask(question):
    print(f"\n❓ {question}")
    print("-"*50)
    result = ask_my_documents(question)
    print(f"💬 {result['answer'][:500]}..." if len(result['answer']) > 500 else f"💬 {result['answer']}")
    return result

# Try different questions
ask("What are the challenges in qualitative research?")

In [ ]:
ask("How do you analyze interview data?")

In [ ]:
ask("What is thematic analysis?")

In [ ]:
# Test with an irrelevant question
ask("What is the capital of France?")

### 💡 Notice the Last Response!

The AI correctly says it doesn't have information about France in the documents. This is RAG working correctly!

- **Without RAG:** AI might hallucinate an answer based on its training
- **With RAG:** AI only answers based on YOUR documents

---

## Step 6: View the Retrieved Context

In [ ]:
# See exactly what context was sent to the AI
question = "What is coding in qualitative research?"
result = ask_my_documents(question)

print("🔍 Context sent to AI:")
print("="*60)
print(result['context'])
print("="*60)

print("\n💬 Answer:")
print(result['answer'])

### 💡 Transparency in RAG

One benefit of RAG is **transparency**:
- You can see exactly what context was used
- You know which documents contained the answer
- You can verify the AI's response against the sources

---

## 🧪 Your Turn: Ask Your Own Questions!

In [ ]:
# ✏️ Try your own question here!
my_question = "YOUR QUESTION HERE"

result = ask_my_documents(my_question)
print(f"❓ {my_question}\n")
print(f"💬 {result['answer']}")

---

## 🎓 Congratulations! You Built a RAG System!

### What You Accomplished:

1. **📄 Document Loading** - Read PDFs into Python
2. **✂️ Text Chunking** - Split into searchable pieces
3. **🔢 Embeddings** - Convert text to vectors
4. **🗄️ Vector Store** - Store in ChromaDB
5. **🤖 AI Integration** - Connect to Groq API
6. **❓ Question Answering** - Full RAG pipeline!

### Key Takeaways:

- **RAG = Retrieval + Generation**
- **Vector search** finds relevant content
- **AI generates** answers from that content
- **Sources are traceable** - you know where answers come from

### Next Steps:

- Add your own PDFs to the `data/` folder
- Experiment with different chunk sizes
- Try different AI models
- Build a web interface with Streamlit or Gradio

---

**Happy Building! 🚀**